## 6.10 Homework

The goal of this homework is to create a tree-based regression model for prediction apartment prices (column `'price'`).

In this homework we'll again use the New York City Airbnb Open Data dataset - the same one we used in homework 2 and 3.

You can take it from [Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

Let's load the data:

In [67]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

%matplotlib inline

In [2]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]

df = pd.read_csv('AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

In [3]:
df.head()

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,40.80902,-73.94190,Private room,150,3,0,0.00,1,365
3,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


* Apply the log tranform to `price`
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1

In [8]:
df["price"] = np.log1p(df["price"])
df["price"]

0        5.010635
1        5.420535
2        5.017280
3        4.499810
4        4.394449
           ...   
48890    4.262680
48891    3.713572
48892    4.753590
48893    4.025352
48894    4.510860
Name: price, Length: 48895, dtype: float64

In [16]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [17]:
y_full_train = df_full_train["price"]
y_train = df_train["price"]
y_val = df_val["price"]
y_test = df_test["price"]

del df_full_train["price"]
del df_train["price"]
del df_val["price"]
del df_test["price"]

Now, use `DictVectorizer` to turn train and validation into matrices:

In [18]:
dicts_full_train = df_full_train.to_dict(orient="records")
dicts_train = df_train.to_dict(orient="records")
dicts_val = df_val.to_dict(orient="records")
dicts_test = df_test.to_dict(orient="records")

In [22]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts_train)
X_val = dv.transform(dicts_val)

## Question 1

Let's train a decision tree regressor to predict the price variable. 

* Train a model with `max_depth=1`

In [27]:
model1 = DecisionTreeRegressor(max_depth=1)
model1.fit(X_train, y_train)
print(export_text(model1, feature_names=dv.get_feature_names()))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [4.29]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [5.15]



Which feature is used for splitting the data?

* `room_type`
* `neighbourhood_group`
* `number_of_reviews`
* `reviews_per_month`

## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1`  (optional - to make training faster)

In [34]:
model2 = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_val)
print("RMSE=", round(mean_squared_error(y_pred, y_val)**0.5, 3))

RMSE= 0.462


What's the RMSE of this model on validation?

* 0.059
* 0.259
* 0.459
* 0.659

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10
* Set `random_state` to `1`
* Evaluate the model on the validation dataset

In [37]:
for n in range(10, 210, 10):
    model2 = RandomForestRegressor(n_estimators=n, n_jobs=-1, random_state=1)
    model2.fit(X_train, y_train)
    y_pred = model2.predict(X_val)
    print(f"n={n}, RMSE={round(mean_squared_error(y_pred, y_val)**0.5, 5)}")

n=10, RMSE=0.46156
n=20, RMSE=0.44818
n=30, RMSE=0.44553
n=40, RMSE=0.44364
n=50, RMSE=0.44233
n=60, RMSE=0.44164
n=70, RMSE=0.44124
n=80, RMSE=0.44113
n=90, RMSE=0.44055
n=100, RMSE=0.43997
n=110, RMSE=0.43947
n=120, RMSE=0.43924
n=130, RMSE=0.4393
n=140, RMSE=0.43901
n=150, RMSE=0.4389
n=160, RMSE=0.4387
n=170, RMSE=0.43863
n=180, RMSE=0.43877
n=190, RMSE=0.43872
n=200, RMSE=0.43875


After which value of `n_estimators` does RMSE stop improving?

- 10
- 50
- 70
- 120

Answer: 120

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`

In [38]:
max_depths = []
ns = []
RMSEs = []
for max_depth in [10, 15, 20, 25]:
    for n in range(10, 210, 10):
        model2 = RandomForestRegressor(max_depth=max_depth, n_estimators=n, n_jobs=-1, random_state=1)
        model2.fit(X_train, y_train)
        y_pred = model2.predict(X_val)
        max_depths.append(max_depth)
        ns.append(n)
        RMSEs.append(mean_squared_error(y_pred, y_val))

In [40]:
params_df = pd.DataFrame()
params_df["max_depth"] = max_depths
params_df["n"]=ns
params_df["RMSE"]=RMSEs


In [50]:
params_df.groupby("max_depth").agg({"RMSE":"min"})

,RMSE
max_depth,
10,0.193269
15,0.190137
20,0.191482
25,0.192453


What's the best `max_depth`:

* 10
* 15
* 20
* 25

Bonus question (not graded):

Will the answer be different if we change the seed for the model?

In [51]:
max_depths = []
ns = []
RMSEs = []
for max_depth in [10, 15, 20, 25]:
    for n in range(10, 210, 10):
        model3 = RandomForestRegressor(max_depth=max_depth, n_estimators=n, n_jobs=-1, random_state=2)
        model3.fit(X_train, y_train)
        y_pred = model3.predict(X_val)
        max_depths.append(max_depth)
        ns.append(n)
        RMSEs.append(mean_squared_error(y_pred, y_val))
        
params_df = pd.DataFrame()
params_df["max_depth"] = max_depths
params_df["n"]=ns
params_df["RMSE"]=RMSEs

params_df.groupby("max_depth").agg({"RMSE":"min"})

,RMSE
max_depth,
10,0.192976
15,0.189576
20,0.191331
25,0.192277


Judging from this single attempt, no; the RMSEs changed a bit, which makes sense, but 15 was still the best max_depth

## Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doint it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the `feature_importances_` field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model

In [53]:
model4 = RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)
model4.fit(X_train, y_train)


RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [66]:
feat_imp_df = pd.DataFrame()

feat_imp_df["feature"] = dv.get_feature_names()
feat_imp_df["feature_importance"] = np.around(model4.feature_importances_, 4)

feat_imp_df.sort_values("feature_importance", ascending=False)

,feature,feature_importance
12,room_type=Entire home/apt,0.3919
3,longitude,0.1541
2,latitude,0.1528
0,availability_365,0.0763
11,reviews_per_month,0.0544
4,minimum_nights,0.0533
10,number_of_reviews,0.0416
7,neighbourhood_group=Manhattan,0.0340
1,calculated_host_listings_count,0.0301
14,room_type=Shared room,0.0050


What's the most important feature? 

* `neighbourhood_group=Manhattan`
* `room_type=Entire home/apt`	
* `longitude`
* `latitude`

Answer: `room_type=Entire home/apt`

## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

In [71]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

feature_names = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=feature_names)

watchlist = [(dtrain, "train"), (dval, "val")]

model5 = xgb.train(xgb_params, 
                   dtrain, 
                   evals=watchlist, 
                   num_boost_round=100)

[0]	train-rmse:3.02752	val-rmse:3.02415
[1]	train-rmse:2.14667	val-rmse:2.14390
[2]	train-rmse:1.53878	val-rmse:1.53721
[3]	train-rmse:1.12557	val-rmse:1.12523
[4]	train-rmse:0.85100	val-rmse:0.85174
[5]	train-rmse:0.67490	val-rmse:0.67752
[6]	train-rmse:0.56687	val-rmse:0.57148
[7]	train-rmse:0.50448	val-rmse:0.51139
[8]	train-rmse:0.46913	val-rmse:0.47777
[9]	train-rmse:0.45009	val-rmse:0.45965
[10]	train-rmse:0.43912	val-rmse:0.44981
[11]	train-rmse:0.43327	val-rmse:0.44475
[12]	train-rmse:0.42936	val-rmse:0.44210
[13]	train-rmse:0.42668	val-rmse:0.44038
[14]	train-rmse:0.42463	val-rmse:0.43943
[15]	train-rmse:0.42259	val-rmse:0.43827
[16]	train-rmse:0.42113	val-rmse:0.43772
[17]	train-rmse:0.42074	val-rmse:0.43787
[18]	train-rmse:0.41896	val-rmse:0.43744
[19]	train-rmse:0.41812	val-rmse:0.43726
[20]	train-rmse:0.41716	val-rmse:0.43691
[21]	train-rmse:0.41499	val-rmse:0.43645
[22]	train-rmse:0.41437	val-rmse:0.43611
[23]	train-rmse:0.41403	val-rmse:0.43614
[24]	train-rmse:0.41391	va

Now change `eta` first to `0.1` and then to `0.01`

In [72]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

feature_names = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=feature_names)

watchlist = [(dtrain, "train"), (dval, "val")]

model5 = xgb.train(xgb_params, 
                   dtrain, 
                   evals=watchlist, 
                   num_boost_round=100)

[0]	train-rmse:3.87217	val-rmse:3.86889
[1]	train-rmse:3.49150	val-rmse:3.48840
[2]	train-rmse:3.14949	val-rmse:3.14635
[3]	train-rmse:2.84232	val-rmse:2.83951
[4]	train-rmse:2.56650	val-rmse:2.56412
[5]	train-rmse:2.31905	val-rmse:2.31692
[6]	train-rmse:2.09714	val-rmse:2.09526
[7]	train-rmse:1.89834	val-rmse:1.89663
[8]	train-rmse:1.72033	val-rmse:1.71878
[9]	train-rmse:1.56120	val-rmse:1.55976
[10]	train-rmse:1.41910	val-rmse:1.41786
[11]	train-rmse:1.29248	val-rmse:1.29149
[12]	train-rmse:1.17977	val-rmse:1.17907
[13]	train-rmse:1.07974	val-rmse:1.07936
[14]	train-rmse:0.99113	val-rmse:0.99118
[15]	train-rmse:0.91299	val-rmse:0.91348
[16]	train-rmse:0.84421	val-rmse:0.84524
[17]	train-rmse:0.78390	val-rmse:0.78525
[18]	train-rmse:0.73111	val-rmse:0.73308
[19]	train-rmse:0.68507	val-rmse:0.68776
[20]	train-rmse:0.64528	val-rmse:0.64883
[21]	train-rmse:0.61109	val-rmse:0.61518
[22]	train-rmse:0.58175	val-rmse:0.58648
[23]	train-rmse:0.55655	val-rmse:0.56186
[24]	train-rmse:0.53529	va

In [73]:
xgb_params = {
    'eta': 0.01, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

feature_names = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=feature_names)

watchlist = [(dtrain, "train"), (dval, "val")]

model5 = xgb.train(xgb_params, 
                   dtrain, 
                   evals=watchlist, 
                   num_boost_round=100)

[0]	train-rmse:4.25336	val-rmse:4.25010
[1]	train-rmse:4.21141	val-rmse:4.20815
[2]	train-rmse:4.16988	val-rmse:4.16661
[3]	train-rmse:4.12877	val-rmse:4.12551
[4]	train-rmse:4.08807	val-rmse:4.08481
[5]	train-rmse:4.04779	val-rmse:4.04454
[6]	train-rmse:4.00792	val-rmse:4.00467
[7]	train-rmse:3.96845	val-rmse:3.96521
[8]	train-rmse:3.92937	val-rmse:3.92615
[9]	train-rmse:3.89070	val-rmse:3.88749
[10]	train-rmse:3.85242	val-rmse:3.84921
[11]	train-rmse:3.81452	val-rmse:3.81133
[12]	train-rmse:3.77701	val-rmse:3.77382
[13]	train-rmse:3.73988	val-rmse:3.73671
[14]	train-rmse:3.70313	val-rmse:3.69996
[15]	train-rmse:3.66674	val-rmse:3.66359
[16]	train-rmse:3.63073	val-rmse:3.62759
[17]	train-rmse:3.59508	val-rmse:3.59195
[18]	train-rmse:3.55979	val-rmse:3.55666
[19]	train-rmse:3.52487	val-rmse:3.52175
[20]	train-rmse:3.49030	val-rmse:3.48719
[21]	train-rmse:3.45608	val-rmse:3.45298
[22]	train-rmse:3.42220	val-rmse:3.41910
[23]	train-rmse:3.38867	val-rmse:3.38559
[24]	train-rmse:3.35548	va

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* 0.01

Answer: 0.1

## Submit the results


Submit your results here: https://forms.gle/wQgFkYE6CtdDed4w8

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline


The deadline for submitting is 20 October 2021, 17:00 CET (Wednesday). After that, the form will be closed.

